In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm

In [2]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.Resize((70,70)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
dataset = torchvision.datasets.ImageFolder(root="../input/persian-mnist/MNIST_persian",transform=transform)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
test = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,(3,3),(1,1),(1,1))
        self.conv2 = nn.Conv2d(32,64,(3,3),(1,1),(1,1))
        self.conv3 = nn.Conv2d(64,128,(3,3),(1,1),(1,1))
        self.conv4 = nn.Conv2d(128,256,(3,3),(1,1),(1,1))
        
        self.fully_connect1 = nn.Linear(256*4*4,256)
        self.fully_connect2 = nn.Linear(256,128)
        self.fully_connect3 = nn.Linear(128,10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,kernel_size=(2,2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,kernel_size=(2,2))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,kernel_size=(2,2))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x,kernel_size=(2,2))
        x = torch.flatten(x,start_dim=1)
        x = F.relu(self.fully_connect1(x))
        x = torch.dropout(x,0.2,train=True)
        x = F.relu(self.fully_connect2(x))
        x = torch.dropout(x,0.5,train=True)
        x = F.softmax(self.fully_connect3(x),dim=1)
        
        return x

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
model.train = True

In [10]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_function = nn.CrossEntropyLoss()

In [11]:
def accuracy(preds,labels):
    _,pred_max = torch.max(preds,1)
    acc = torch.sum(pred_max==labels,dtype=torch.float64) / len(preds)
    
    return acc

In [12]:
for epoch in range(30):
    train_loss = 0.0
    train_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0
    for image, label in tqdm(train):
        image.to(device)
        label.to(device)
        optimizer.zero_grad()
        
        predict = model(image)
        loss = loss_function(predict,label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss
        train_acc += accuracy(predict,label)
    
    for image, label in test:
        image.to(device)
        label.to(device)
        predict = model(image)
        loss = loss_function(predict,label)
        
        test_loss += loss
        test_acc += accuracy(predict,label)
    
    total_train_loss = train_loss / len(train)
    total_train_acc = train_acc / len(train)
    total_test_loss = test_loss / len(test)
    total_test_acc = test_acc / len(test)
    
    print(f"Epochs: {epoch+1}, Accuracy: {total_train_acc}, Loss: {total_train_loss}")
    print(f"Epochs: {epoch+1}, Val_Accuracy: {total_test_acc}, Val_Loss: {total_test_loss}")

100%|██████████| 38/38 [00:13<00:00,  2.91it/s]


Epochs: 1, Accuracy: 0.09950657894736842, Loss: 2.3028862476348877
Epochs: 1, Val_Accuracy: 0.1015625, Val_Loss: 2.303030014038086


100%|██████████| 38/38 [00:08<00:00,  4.27it/s]


Epochs: 2, Accuracy: 0.10115131578947369, Loss: 2.302900791168213
Epochs: 2, Val_Accuracy: 0.1015625, Val_Loss: 2.3029346466064453


100%|██████████| 38/38 [00:07<00:00,  4.91it/s]


...

Epochs: 28, Accuracy: 0.9366776315789473, Loss: 1.5246905088424683
Epochs: 28, Val_Accuracy: 0.984375, Val_Loss: 1.4792197942733765


100%|██████████| 38/38 [00:08<00:00,  4.41it/s]


Epochs: 29, Accuracy: 0.9564144736842105, Loss: 1.5050941705703735
Epochs: 29, Val_Accuracy: 0.96875, Val_Loss: 1.4970831871032715


100%|██████████| 38/38 [00:08<00:00,  4.23it/s]


Epochs: 30, Accuracy: 0.953125, Loss: 1.5100843906402588
Epochs: 30, Val_Accuracy: 0.9609375, Val_Loss: 1.5009663105010986


In [13]:
torch.save(model.state_dict(),"weights.pth")